<a href="https://colab.research.google.com/github/ayushmangupta/DataScience/blob/master/tfCustumLayers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install tensorflow-gpu==2.0.0-beta1
#!pip install tensorflow==2.0.0-beta1
#!pip install "tensorflow_hub==0.4.0"
%matplotlib inline

import tensorflow_hub as hub
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import gym
tfk = tf.keras
tfkl = tf.keras.layers
tf.__version__


'2.0.0-beta1'

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print("Shapes Before: ",x_train.shape)
#y_train = y_train.astype(np.float64)
#y_test = y_test.astype(np.float64)

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
print("Shapes After: ",x_train.shape)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Shapes Before:  (60000, 28, 28)
Shapes After:  (60000, 28, 28, 1)


In [0]:
class Net2(tfk.Model):
  def __init__(self):
    super(Net2, self).__init__()
    
    self.flattern= tfkl.Flatten()
    self.conv = tfkl.Conv2D(32,kernel_size=3,activation="relu")
    #self.dropout = tfkl.Dropout(0.2)
    self.d2 = tfkl.Dense(128,activation="relu")
    self.d3 = tfkl.Dense(10,activation= 'softmax')
    
    
  def call(self,input):
    input = tf.convert_to_tensor(input,dtype=tf.float32)
    out = self.conv(input)
    #out = self.dropout(out)
    #out = self.batchnorm(out)
    out = self.flattern(out)
    out = self.d2(out)
    return self.d3(out)
    #print("hid2",out)
    #return self.outputs(out)
    
def get_model():
  return Net2()
model = Net2() 


In [0]:
# #@title Default title text
# class Net1(tfk.Model):
#   def __init__(self):
#     super(Net1, self).__init__()
    
#     self.flattern= tfkl.Flatten(input_shape=(28,28))
#     self.d1 = tfkl.Dense(64,activation="relu")
#     self.backprob = tfkl.BatchNormalization()
#     self.dropout = tfkl.Dropout(0.2)
#     self.d2 = tfkl.Dense(64,activation="relu")
#     self.d3 = tfkl.Dense(10,activation= 'softmax')
    
    
#   def call(self,input):
#     input = tf.convert_to_tensor(input,dtype=tf.float64)
#     out = self.flattern(input)
#     out = self.d1(out)
#     out = self.dropout(out)
#     #out = self.backprob(out)
#     out = self.d2(out)
#     return self.d3(out)
#     #print("hid2",out)
#     #return self.outputs(out)
   
    
    
   
    

In [0]:
loss_func = tfk.losses.SparseCategoricalCrossentropy()
optimizer = tfk.optimizers.Adam(lr=0.001)

In [0]:
train_loss = tfk.metrics.Mean(name="train_loss")
test_loss = tfk.metrics.Mean(name="test_loss")

train_accuracy = tfk.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy = tfk.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
#@tf.function
def train_step(images,labels):
  with tf.GradientTape() as tape:
    prediction = model(images)
    losses = loss_func(labels,prediction)
    gradients = tape.gradient(losses,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    train_loss(losses)
    train_accuracy(labels,prediction)
    #print(train_loss.result())

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  #print("pred",predictions)
  t_loss = loss_func(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)
  #print("testLoss",test_loss.result())

In [0]:
tf.test.is_gpu_available()

True

In [0]:
EPOCHS = 2

for epoch in range(EPOCHS):
  #print("Epoch ",epoch)
  for images, labels in tqdm(train_ds):
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
  #template = 'Epoch {}, Loss: {}, Accuracy: {}'
  #print (template.format(epoch+1,train_loss.result(),train_accuracy.result()*100))

0it [00:00, ?it/s]WARNING: Logging before flag parsing goes to stderr.
W0616 11:58:50.761923 140719912593280 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1220: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1875it [00:45, 41.12it/s]
0it [00:00, ?it/s]

Epoch 1, Loss: 0.13167384266853333, Accuracy: 96.00499725341797, Test Loss: 0.07399452477693558, Test Accuracy: 97.68000030517578


1875it [00:42, 43.76it/s]


Epoch 2, Loss: 0.08526760339736938, Accuracy: 97.38833618164062, Test Loss: 0.06854066997766495, Test Accuracy: 97.82500457763672


In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.astype('float32') / 255, x_test.astype('float32') / 255
print("Shapes Before: ",x_train.shape)
#y_train = y_train.astype(np.float64)
#y_test = y_test.astype(np.float64)

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
print("Shapes After: ",x_train.shape)




Shapes Before:  (60000, 28, 28)
Shapes After:  (60000, 28, 28, 1)


In [0]:
model1 = get_model()
model1.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop())
history = model1.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

Train on 60000 samples
60000/60000 [==============================] - 4s 60us/sample - loss: 0.1532


In [0]:
model1.summary()

Model: "net2_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          multiple                  0         
_________________________________________________________________
conv2d_9 (Conv2D)            multiple                  320       
_________________________________________________________________
dense_18 (Dense)             multiple                  2769024   
_________________________________________________________________
dense_19 (Dense)             multiple                  1290      
Total params: 2,770,634
Trainable params: 2,770,634
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.save_weights("modelweght2a",save_format='tf')

In [0]:
ls -lah

total 106M
drwxr-xr-x 1 root root 4.0K Jun 16 12:02 ./
drwxr-xr-x 1 root root 4.0K Jun 16 10:52 ../
-rw-r--r-- 1 root root 1.9K Jun 16 11:46 .5.data-00000-of-00002
-rw-r--r-- 1 root root  22M Jun 16 11:46 .5.data-00001-of-00002
-rw-r--r-- 1 root root 1.3K Jun 16 11:46 .5.index
-rw-r--r-- 1 root root   81 Jun 16 12:02 checkpoint
drwxr-xr-x 1 root root 4.0K Jun 12 16:04 .config/
-rw-r--r-- 1 root root  597 Jun 16 11:28 hey.data-00000-of-00002
-rw-r--r-- 1 root root  22M Jun 16 11:28 hey.data-00001-of-00002
-rw-r--r-- 1 root root  22M Jun 16 11:28 hey.h5
-rw-r--r-- 1 root root  521 Jun 16 11:28 hey.index
-rw-r--r-- 1 root root 1.9K Jun 16 11:47 model1weght.data-00000-of-00002
-rw-r--r-- 1 root root  22M Jun 16 11:47 model1weght.data-00001-of-00002
-rw-r--r-- 1 root root 1.3K Jun 16 11:47 model1weght.index
-rw-r--r-- 1 root root  602 Jun 16 12:02 modelweght2a.data-00000-of-00002
-rw-r--r-- 1 root root  22M Jun 16 12:02 modelweght2a.data-00001-of-00002
-rw-r--r-- 1 root root  521 Jun 16 12:

In [0]:
#loading model

In [0]:
model2 =get_model()
model2.load_weights('modelweght2a')

In [0]:
model2.predict(x_test)

TypeError: ignored

In [0]:
model2.build(input_shape=(None,28,28,1))

In [0]:
model2.summary()

Model: "net2_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


ValueError: ignored

ValueError: ignored